# Coalesce 2022 Python models on Databricks workshop

**Important**: replace the schema below with your own.

In [0]:
# IMPORTANT: replace the schema
schema = "dbt_cody"

In [0]:
# "spark" is the canonical magical Spark session object
# variety is the spice of life
session = spark
s = session

## Level 1: Describe the orders table

Compute summary statistics about the orders table.

Hint: check out the `describe()` method from pandas (on Spark). Do you want to work with PySpark DataFrames, or pandas on Spark DataFrames?

In [0]:
orders = s.table(f"{schema}.orders")
orders

In [0]:
type(orders)

## Level 2: Pivot the orders table

Orders contains most important details, but is lacking the breakdown by product_id. Add the subtotal for each product_id to the orders table.

Hint: this can be achieved in dataframes generally via `pivot` methods. Can you find a better way?

In [0]:
stg_products = s.table(f"{schema}.stg_products")
stg_products

In [0]:
stg_order_items = s.table(f"{schema}.stg_order_items")
stg_order_items

## Level 3: Cluster with KMeans

You suspect there are N personas representing customers. Attempt to group the data from "Level 2" into N clusters so that it can be further analyzed.

Hint: create a `X` array for use in KMeans and PCA below.

In [0]:
# use the dataframe object from above, or cheat and use the pivot_py model from an already built schema
orders_with_subtotals = s.table(f"{schema}.pivot_py")
orders_with_subtotals

In [0]:
import pyspark.pandas as ps

from sklearn.cluster import KMeans

In [0]:
n_clusters = 5

In [0]:
X = None # engineer features for the KMeans model
cluster_labels = None # predict from the KMeans model

In [0]:
# visualize the clusters

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

n_components = 3
pca = PCA(n_components=n_components)
pca = pca.fit(X)
X_pca = pca.transform(X)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection="3d")

ax.scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    X_pca[:, 2],
    c=cluster_labels, # how do you get these?
)

In [0]:
# example of joining the KMeans predictions with the original dataframe
# not necessarily working. create X, get the predictions, then 
orders_with_subtotals_and_clusters = orders_with_subtotals.merge(
    ps.DataFrame(data=cluster_labels, columns=["cluster_label"]),
    left_index=True,
    right_index=True

## Level 4: Forecast with Prophet

Forecast revenue with Prophet models for each location_id.

In [0]:
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

In [0]:
revenue = s.table(f"{schema}.revenue_weekly_by_location").pandas_api()
revenue.head()

In [0]:
# at the end, something like this should work to plot the forecasts
# note that `.plot(...)` here is a Prophet model's method
for location in locations:
    models[location].plot(forecasts[location])
    plt.title(location)

Now how do you run these models in dbt? Can you save them in one (dbt) model, and load them in another?